In [1]:
from datetime import datetime
import os
import time
start_time = time.time()
now = datetime.now()
folderdir = now.strftime("%D_%H_%M").replace('/','')
print("Current Time =", folderdir)
os.makedirs(folderdir) 

Current Time = 031020_00_17


In [2]:
import local_load_data as ldata
import local_param as pm
import clean_data as cdata
import warnings
warnings.filterwarnings("ignore")
data_All_years = ldata.start_load(pm.csv_file)
#data_years = ldata.get_TrafficData_CSV_year(data_All_years,pm.y_start,pm.y_end)


Total Population involed in all traffic accidents:  6772563

Total number of all traffic accidents:  2570235


In [3]:
data_cleaned = cdata.data_clean_columns(data_All_years,pm.columns_clean,v_num=pm.v_num)
data_cleaned = cdata.modify_data(data_cleaned,pm.columns_grp)
cdata.get_data_stats(data_cleaned,pm.columns_stats)

C_YEAR column data is distributed as below: 

2015    0.079153
2016    0.077969
2010    0.067508
2011    0.067434
2012    0.067217
2004    0.065796
2017    0.060142
2008    0.058724
2014    0.058643
2003    0.051425
2013    0.051214
2009    0.049559
2007    0.049163
2001    0.041607
2002    0.041177
2005    0.041134
2006    0.040887
2000    0.015955
1999    0.015293
Name: C_YEAR, dtype: float64
Total size : (1397617,)
C_VEHS column data is distributed as below: 

2    0.777808
1    0.222192
Name: C_VEHS, dtype: float64
Total size : (1397617,)
C_RSUR column data is distributed as below: 

1    0.679174
2    0.195049
5    0.056241
3    0.046207
4    0.016623
6    0.005491
7    0.000946
8    0.000212
9    0.000057
Name: C_RSUR, dtype: float64
Total size : (1397617,)
C_RALN column data is distributed as below: 

1    0.760717
2    0.101622
3    0.078927
4    0.044743
5    0.008082
6    0.005909
Name: C_RALN, dtype: float64
Total size : (1397617,)
C_WTHR column data is distributed as below:

In [4]:
import creat_ML_data as MLdata
data_selected = MLdata.create_data_for_ML(data_cleaned, pm.traffic_data_headers, pm.dummy_fields, pm.regression)

In [5]:
data_selected = MLdata.create_data_for_ML(data_cleaned, pm.traffic_data_headers, pm.dummy_fields, pm.regression)
test_data, train_x, test_x, train_y, test_y = MLdata.train_test_data(data_selected,pm.time,pm.test_size,pm.year,pm.interval,pm.traffic_data_headers)
from sklearn.utils import class_weight
import numpy as np
class_weight.compute_class_weight('balanced', np.unique(train_y), train_y)

array([ 0.85299215,  0.55397938, 44.37556164])

In [ ]:
import LR_model_present as LRmp
import Tree_model_present as Tmp
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
if pm.regression =='lr':
  file_name = 't_{}_tst{}_trn{}_Csize_{}_pnt_{}_cw_{}_mltc_{}_svr_{}_cv_{}'.format(pm.time,pm.year,pm.interval,len(pm.C),pm.penalty,pm.classweight,pm.multi_class,pm.solver,pm.cv)
  result, all_model = LRmp.build_model_LR(train_x,train_y,test_x,test_y,pm.C,pm.penalty,pm.classweight,pm.multi_class,pm.solver,pm.cv)
elif pm.regression =='tree':
  file_name = 't_{}_tst{}_trn{}_dep_{}_splt_{}_crt_{}_cw_{}'.format(pm.time,pm.year,pm.interval,pm.max_dep,pm.split_type,pm.crt_type,pm.classweight)
  result, all_model = Tmp.build_model_tree(train_x,train_y,test_x,test_y,pm.max_dep,pm.split_type,pm.crt_type,pm.classweight)

1：No Injury
 2:Injury
 3:Fatal

p:  0, C=10  ,cv=5


In [ ]:
import get_models_parameter as gmp
gmp.get_models_parameter(all_model,pm.regression,test_data)

In [ ]:
orders = ['Accuracy']  + [col for col in result if col != 'Accuracy']
result = result[orders]
file_name = '{}\t_{}_tst{}_trn{}_Csize_{}_pnt_{}_cw_{}_mltc_{}_svr_{}_cv_{}'.format(folderdir,pm.time,pm.year,pm.interval,len(pm.C),pm.penalty,pm.classweight,pm.multi_class,pm.solver,pm.cv)
result.to_csv(file_name+pm.regression+'_accuracy_result'+'.csv') 
import local_Result_Analysis as RA
allmodel_Acc_error_mean_sqrt,allmodel_victim_error_mean_sqrt,allmodel_Acc_error_mean,allmodel_victim_error_mean = RA.compare_model(test_data, test_x, all_model,pm.money_path)
allmodel_Acc_error_mean_sqrt.to_csv(file_name+'allmodel_Acc_error_mean_sqrt.csv')
allmodel_victim_error_mean_sqrt.to_csv(file_name+'allmodel_victim_error_mean_sqrt.csv')
allmodel_Acc_error_mean.to_csv(file_name+'allmodel_Acc_error_mean.csv')
allmodel_victim_error_mean.to_csv(file_name+'allmodel_victim_error_mean.csv')

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))